# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

## Dataset

### Overview
Predicting the  price of the used bmw cars based on transmission, mileage, fuel type, road tax, miles per gallon (mpg), and engine size

Dataset is gathered from https://www.kaggle.com/adityadesai13/used-car-dataset-ford-and-mercedes?select=bmw.csv

In [1]:
import azureml.core
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset
from azureml.core.workspace import Workspace
from azureml.core import Experiment, Webservice, Model
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice


import pandas as pd
import sklearn
import json

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'bmw_exp'

experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
bmw_exp,ud-ws,Link to Azure Machine Learning studio,Link to Documentation


In [3]:
cluster_name="ud-cluster"

try:
    cpu_cluster=ComputeTarget(workspace=ws, name=cluster_name)
    print("Existing cluster detected, make use of it!")
except ComputeTargetException:
    print("New compute cluster creation in progress...")
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)   
    
cpu_cluster.wait_for_completion(show_output=True)         
print("Cluster is ready")

Existing cluster detected, make use of it!
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
Cluster is ready


## Dataset

In [4]:
from azureml.core import Dataset
dataset_name = 'bmw_cars'

# Get a dataset by name
bmw_cars = Dataset.get_by_name(workspace=ws, name=dataset_name)

# Load a TabularDataset into pandas DataFrame
df = bmw_cars.to_pandas_dataframe()
df.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,5 Series,2014,11200,Automatic,67068,Diesel,125,57.6,2.0
1,6 Series,2018,27000,Automatic,14827,Petrol,145,42.8,2.0
2,5 Series,2016,16000,Automatic,62794,Diesel,160,51.4,3.0
3,1 Series,2017,12750,Automatic,26676,Diesel,145,72.4,1.5
4,7 Series,2014,14500,Automatic,39554,Diesel,160,50.4,3.0


## AutoML Configuration
Choosen normalized_root_mean_squared_error as the primary metric as this is a regression problem.


In [5]:
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 10,
    "primary_metric" : 'normalized_root_mean_squared_error',
    "n_cross_validations": 5
}
automl_config = AutoMLConfig(compute_target=cluster_name,
                             task = "regression",
                             training_data=bmw_cars,
                             label_column_name="price", 
                             enable_early_stopping= True,
                             featurization= 'auto',
                             enable_voting_ensemble= True,
                             **automl_settings
                            )

In [6]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

*

{'runId': 'AutoML_4812c573-77c1-4c32-8929-e2a949fdfa03',
 'target': 'ud-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-02-23T18:29:35.262386Z',
 'endTimeUtc': '2021-02-23T19:00:13.387226Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'normalized_root_mean_squared_error',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'ud-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"bmw_exp","subscription_id":"d7a31e64-085f-42e0-bb3e-c1135731e46a","resource_group":"ud-rg","workspace_name":"ud-ws","region":"westus2","compute_target":"ud-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"normalized_root_mean_squared_error","task_type":"regression","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":null,"fea

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [8]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

Package:azureml-automl-runtime, training version:1.22.0, current version:1.20.0
Package:azureml-core, training version:1.22.0, current version:1.20.0
Package:azureml-dataprep, training version:2.9.1, current version:2.7.3
Package:azureml-dataprep-native, training version:29.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.7.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.22.0, current version:1.20.0
Package:azureml-defaults, training version:1.22.0, current version:1.20.0
Package:azureml-interpret, training version:1.22.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.22.0, current version:1.20.0
Package:azureml-telemetry, training version:1.22.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.22.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.22.0, current version:1.20.0


Run(Experiment: bmw_exp,
Id: AutoML_4812c573-77c1-4c32-8929-e2a949fdfa03_59,
Type: azureml.scriptrun,
Status: Completed)
RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=None,
                                                             enable_feature_sweeping=None,
                                                             feature_sweeping_config=None,
                                                             feature_sweeping_timeout=None,
                                                             featurization_config=None,
                                                             force_text_dnn=None,
                                                             is_cross_validation=None,
                                                             is_onnx_compatible=None,
                                                             logger=No

In [9]:
#TODO: Save the best model
best_run.register_model(model_name='automl_model',model_path='/outputs')

Model(workspace=Workspace.create(name='ud-ws', subscription_id='d7a31e64-085f-42e0-bb3e-c1135731e46a', resource_group='ud-rg'), name=automl_model, id=automl_model:1, version=1, tags={}, properties={})

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service